In [115]:
import cv2
import numpy as np

def out(img,nam="img",close = 1):
    cv2.imshow(nam,img)
    cv2.waitKey(0)
    if(close == 1):
        cv2.destroyWindow(nam)
    return

def load_img():
    img = cv2.imread("target_obj.jpg")
    return img

In [2]:
cap = cv2.VideoCapture(0)

In [84]:
test = cv2.imread("basic_test_case.jpg")
test_good = cv2.fastNlMeansDenoisingColored(test)

In [85]:
out(test_good)
gray = cv2.cvtColor(test_good,cv2.COLOR_BGR2GRAY)


In [86]:
blurred = cv2.GaussianBlur(gray,(5,5),0)
b_lap =  cv2.Laplacian(blurred,cv2.CV_64F)

In [87]:
b_fil = cv2.bilateralFilter(gray,9,75,75)

In [88]:
lap = cv2.Laplacian(gray,cv2.CV_64F)

In [89]:
out(lap,"lap",0)

In [90]:
out(b_lap,"b_lap",0)

In [91]:
out(b_fil)

In [92]:
threshold = cv2.adaptiveThreshold(b_fil,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)


In [93]:
out(threshold)

In [94]:
objects_roi = cv2.bitwise_and(b_fil,b_fil,mask=threshold)

In [84]:
out(objects_roi)

In [95]:
cv2.destroyAllWindows()

Target Detection

In [97]:
def trackBall():
    _,img = cap.read()
    _,img = cap.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
    if circles is None:
        cv2.imshow("ball",img)
        cv2.waitKey(0)
        return "No_Object_Found"
    circles = np.round(circles).astype("int")
    for (x,y,r) in circles:
        cv2.circle(img, (x, y), r, (0, 255, 0), 4)
    cv2.imshow("ball",img)
    cv2.waitKey(0)
        
    

In [70]:
# while True:
trackBall()

In [56]:
    _,img = cap.read()
    _,img = cap.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)


In [74]:
cv2.imshow("img",img)

In [75]:
cv2.waitKey(0)

113

In [40]:
img_good = cv2.fastNlMeansDenoisingColored(img)
clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))

In [43]:
#img
gray = cv2.cvtColor(img_good,cv2.COLOR_BGR2GRAY)
good = clahe.apply(gray)

In [108]:
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cir = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1.2,100)

In [109]:
for x,y,r in circles[0]:
    cv2.circle(img,(x,y),r,-1)

In [73]:
_,img = cap.read()
_,img = cap.read()

In [112]:
img = cv2.GaussianBlur(img,(5,5),1.0,1.0)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)


In [113]:
circles = np.round(circles).astype("int")

AttributeError: 'NoneType' object has no attribute 'rint'

In [76]:
cv2.imwrite("target_obj.jpg",img)

True

In [82]:
if circles is None:
    cv2.imshow("ball",img)
    cv2.waitKey(0)
    #return "No_Object_Found"
else:
    circles = np.round(circles[0, :]).astype("int")
    for (x,y,r) in circles:
        cv2.circle(img, (x, y), r, (0, 255, 0), 4)
    out(img)

In [114]:
out(img)

In [116]:
cv2.destroyAllWindows()

In [117]:
cap.release()

# multi-shape object detection

In [1]:
def cv(imgn,color,shape): # get input to dir by providing color and shape
    #imgn = cv2.medianBlur(imgn,3)
    #cv2.imshow("blurred",imgn)
    img = cv2.cvtColor(imgn,cv2.COLOR_BGR2HSV)
    if color == 'red':
        lower_red = np.array([0,50,50])
        upper_red = np.array([10,255,255])
        mask0 = cv2.inRange(img, lower_red, upper_red)
        lower_red = np.array([170,50,50])
        upper_red = np.array([180,255,255])
        mask1 = cv2.inRange(img, lower_red, upper_red)
        mask = cv2.bitwise_or(mask0,mask1)
    elif color =='blue':
        lower = np.array([100,50,50])
        upper = np.array([140,255,255])
        mask =  cv2.inRange(img, lower, upper)
    elif color== 'green':
        lower = np.array([40,50,50])
        upper = np.array([80,255,255])
        mask =  cv2.inRange(img, lower, upper)
    else:
        return -1,-1
    res = cv2.bitwise_and(imgn,imgn, mask= mask)
    #cv2.imshow("result",res)
    cv2.imwrite("color_filter_result.jpg",res)
    
    res_gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    _,thres = cv2.threshold(res_gray,100,255,cv2.THRESH_BINARY)
    #cv2.imshow("threshold",thres)
    cv2.imwrite("thresholded.jpg",thres)
    _,contours, h = cv2.findContours(thres,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    contours= sorted(contours, key = cv2.contourArea, reverse = True) 
    pt = (180, 3 * img.shape[0] // 4)
    im = imgn.copy()
    #cv2.drawContours(im,contours,-1,[255,255,255])
    #cv2.imshow("res",im)
    cv2.imwrite("final_output.jpg",im)
    try:
        for cnt in contours:
            approx = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
            M = cv2.moments(cnt)
            cy = int(M['m10']/M['m00'])
            cx = int(M['m01']/M['m00'])
            #print cx,cy
            if shape == "sphere":
                c = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,param1=50,param2=30,minRadius=0,maxRadius=0)[:1]
                cv2.drawContours(im,[cnt],-1,[255,255,255])
                cv2.circle(im,c,3,(255,255,255),-1)
                cv2.imwrite("final_output.jpg",im)
                return c[0],c[1] 
            
            elif shape =="cube" and len(approx)<8:
                cv2.drawContours(im,[cnt],-1,[255,255,255])
                cv2.circle(im,(cy,cx),3,(255,255,255),-1)
                cv2.imwrite("final_output.jpg",im)
                return cx,cy
            elif shape =="cylinder" and len(approx)>=8:
                cv2.drawContours(im,[cnt],-1,[255,255,255])
                cv2.circle(im,(cy,cx),3,(255,255,255),-1)
                cv2.imwrite("final_output.jpg",im)
                return cx,cy
    except:
            pass
    return -1,-1 

In [2]:
# call format as below

'''
parameters:
Input: 
img - a colour image
color - the colour of the object
shape - the shape of the object

Output:
 cx,cy - (x,y) of centroid of object
 
Call:
cx,cy = cv(img,color,shape)

'''

'\nparameters:\nInput: \nimg - a colour image\ncolor - the colour of the object\nshape - the shape of the object\n\nOutput:\n cx,cy - (x,y) of centroid of object\n \nCall:\ncx,cy = cv(img,color,shape)\n\n'